# Applications a MNIST

In [ ]:
%pylab inline
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
print ("PACKAGES LOADED")

# LOAD MNIST

In [ ]:
mnist = input_data.read_data_sets('data/', one_hot=True)

In [ ]:
# How does the training data look like?
trainimg   = mnist.train.images
trainlabel = mnist.train.labels

print ("How does the training data look like?")
nsample = 5
randidx = random.randint(trainimg.shape[0], size=nsample)


for i in randidx:
    curr_img   = np.reshape(trainimg[i, :], (28, 28)) # 28 by 28 matrix 
    curr_label = np.argmax(trainlabel[i, :] ) # Label
    matshow(curr_img, cmap=plt.get_cmap('gray'))
    title("" + str(i) + "th Training Data " 
              + "Label is " + str(curr_label))
    print ("" + str(i) + "th Training Data " 
           + "Label is " + str(curr_label))

# DEFINE MODEL

In [ ]:
# NETWORK TOPOLOGIES
n_hidden_1 = 256 
n_hidden_2 = 128 
n_input    = 784 
n_classes  = 10  

# INPUTS AND OUTPUTS
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# NETWORK PARAMETERS
stddev = 0.1

w = tf.Variable(tf.random_normal([n_input, n_classes], stddev=stddev))
b = tf.Variable(tf.random_normal([n_classes]))


# Logistic Regression (Reseau de neurones mono-couche)

In [ ]:
def logistic_regression(_X):
    return tf.add(tf.matmul(_X, w),b)

In [ ]:
# PREDICTION
pred = logistic_regression(x)

# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels= y)) 
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 

corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

In [ ]:
# RUN
#PARAMETERS
training_epochs = 20
batch_size      = 100
display_step    = 4
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
print ("OPTIMIZATION FINISHED")

# Reseau de neurones multicouches

In [ ]:
h1  = tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev))
b1  = tf.Variable(tf.random_normal([n_hidden_1]))

h2  = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev))
b2  = tf.Variable(tf.random_normal([n_hidden_2]))

wout= tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
bout= tf.Variable(tf.random_normal([n_classes]))

In [ ]:
def multilayer_perceptron(_X):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, h1),b1)) 
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1,h2), b2))
    return (tf.matmul(layer_2, wout) + bout) 

# DEFINE FUNCTIONS

In [ ]:
# PREDICTION
pred = multilayer_perceptron(x)

# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) 
# optm = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost) 
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

# RUN

In [ ]:
# PARAMETERS
training_epochs = 20
batch_size      = 100
display_step    = 4
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED")